# Project 3

## Reading the credentials in the yaml file

In [1]:
from database_utils import DatabaseConnector

connector = DatabaseConnector()
connector.read_db_creds()




Returning the dictionary of the credentials{'RDS_HOST': 'data-handling-project-readonly.cq2e8zno855e.eu-west-1.rds.amazonaws.com', 'RDS_PASSWORD': 'AiCore2022', 'RDS_USER': 'aicore_admin', 'RDS_DATABASE': 'postgres', 'RDS_PORT': 5432}
Credentials loaded sucessfully!!


## initialising and returning an sqlalchemy database engine.

In [2]:
connection = connector.init_db_engine()

Connection successful!


## Reading the data from the RDS database

In [3]:
connector_tables = connector.list_db_tables()


Tables in the database: ['legacy_store_details', 'dim_card_details', 'legacy_users', 'orders_table']


## extracting the database table to a pandas DataFrame.

In [4]:
from data_extraction import DataExtractor
user_table = 'legacy_users'

extractor = DataExtractor(connection)

user_df = extractor.read_rds_table(user_table)
print(user_df.head(5))
print(user_df.info())

   index first_name last_name date_of_birth                    company  \
0      0   Sigfried     Noack    1990-09-30         Heydrich Junitz KG   
1      1        Guy     Allen    1940-12-01                    Fox Ltd   
2      2      Harry  Lawrence    1995-08-02  Johnson, Jones and Harris   
3      3     Darren   Hussain    1972-09-23                Wheeler LLC   
4      4      Garry     Stone    1952-12-20                 Warner Inc   

                  email_address  \
0             rudi79@winkler.de   
1  rhodesclifford@henderson.com   
2  glen98@bryant-marshall.co.uk   
3    daniellebryan@thompson.org   
4       billy14@long-warren.com   

                                             address         country  \
0                       Zimmerstr. 1/0\n59015 Gießen         Germany   
1  Studio 22a\nLynne terrace\nMcCarthymouth\nTF0 9GH  United Kingdom   
2                 92 Ann drive\nJoanborough\nSK0 6LR  United Kingdom   
3             19 Robinson meadow\nNew Tracy\nW22 2QG  Un

## cleaning data

In [5]:
from data_cleaning import DataCleaning

clean_data = DataCleaning()
clean_df = clean_data.clean_user_data(user_df)



Checking for Null values:
index            0.0
first_name       0.0
last_name        0.0
date_of_birth    0.0
company          0.0
email_address    0.0
address          0.0
country          0.0
country_code     0.0
phone_number     0.0
join_date        0.0
user_uuid        0.0
dtype: float64
Deleting rows with more than 4 Null values
       index first_name last_name date_of_birth                       company  \
0          0   Sigfried     Noack    1990-09-30            Heydrich Junitz KG   
1          1        Guy     Allen    1940-12-01                       Fox Ltd   
2          2      Harry  Lawrence    1995-08-02     Johnson, Jones and Harris   
3          3     Darren   Hussain    1972-09-23                   Wheeler LLC   
4          4      Garry     Stone    1952-12-20                    Warner Inc   
...      ...        ...       ...           ...                           ...   
15315  14913    Stephen   Jenkins    1943-08-09  Thornton, Carroll and Newman   
15316  14994    

## uploading the cleaned dataframe into the database

In [6]:


connector.upload_to_db(connection, clean_df, 'dim_users')


AttributeError: 'NoneType' object has no attribute 'to_sql'